In [1]:
!pip install -r requirements.txt

Defaulting to user installation because normal site-packages is not writeable
   ---------------------------------------- 0.0/2.8 MB ? eta -:--:--
   ----------------------------- ---------- 2.1/2.8 MB 11.8 MB/s eta 0:00:01
   ---------------------------------------- 2.8/2.8 MB 10.9 MB/s  0:00:00
   ---------------------------------------- 0.0/60.7 MB ? eta -:--:--
   - -------------------------------------- 2.4/60.7 MB 12.2 MB/s eta 0:00:05
   --- ------------------------------------ 4.7/60.7 MB 11.9 MB/s eta 0:00:05
   ---- ----------------------------------- 6.3/60.7 MB 11.7 MB/s eta 0:00:05
   ---- ----------------------------------- 6.3/60.7 MB 11.7 MB/s eta 0:00:05
   ---- ----------------------------------- 6.3/60.7 MB 11.7 MB/s eta 0:00:05
   ---- ----------------------------------- 6.3/60.7 MB 11.7 MB/s eta 0:00:05
   ----- ---------------------------------- 8.4/60.7 MB 5.6 MB/s eta 0:00:10
   ------- -------------------------------- 10.7/60.7 MB 6.3 MB/s eta 0:00:08
   ------

In [2]:
!pip install --pre torch torchvision torchaudio --extra-index-url https://download.pytorch.org/whl/nightly/cpu

Looking in indexes: https://pypi.org/simple, https://download.pytorch.org/whl/nightly/cpu


In [2]:
!pip install --upgrade transformers

Defaulting to user installation because normal site-packages is not writeable
   ---------------------------------------- 0.0/11.6 MB ? eta -:--:--
   ------- -------------------------------- 2.1/11.6 MB 10.7 MB/s eta 0:00:01
   --------------- ------------------------ 4.5/11.6 MB 11.2 MB/s eta 0:00:01
   ---------------------- ----------------- 6.6/11.6 MB 11.2 MB/s eta 0:00:01
   ------------------------------ --------- 8.9/11.6 MB 11.1 MB/s eta 0:00:01
   -------------------------------------- - 11.3/11.6 MB 11.2 MB/s eta 0:00:01
   ---------------------------------------- 11.6/11.6 MB 11.0 MB/s  0:00:01
   ---------------------------------------- 0.0/563.4 kB ? eta -:--:--
   ---------------------------------------- 563.4/563.4 kB 9.7 MB/s  0:00:00
   ---------------------------------------- 0.0/2.7 MB ? eta -:--:--
   ----------------------------------- ---- 2.4/2.7 MB 11.2 MB/s eta 0:00:01
   ---------------------------------------- 2.7/2.7 MB 11.0 MB/s  0:00:00

  Attempting uni

In [3]:
# Import all libs
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
from sklearn.metrics import (
    f1_score, recall_score, balanced_accuracy_score,
    matthews_corrcoef, precision_recall_curve, average_precision_score
)
from sklearn.preprocessing import label_binarize
import pandas as pd
import numpy as np
from tqdm.auto import tqdm

In [4]:
# Authenticate with HuggingFace
import os
from dotenv import load_dotenv
from huggingface_hub import login

# from google.colab import userdata
# hugging_face_token = userdata.get("hf_token") #If using gg colab

load_dotenv() #If using VSCode
hugging_face_token = os.getenv("hf_token") #If using VSCode

login(token=hugging_face_token)

Note: Environment variable`HF_TOKEN` is set and is the current active token independently from the token you've just configured.


In [5]:
# Check memory allocated for MPS
if torch.backends.mps.is_available():
    device = torch.device("mps")
    print("MPS device available:", torch.backends.mps.is_available())
    # This tells you memory usage is not directly exposed, but you can monitor allocated tensors
    x = torch.randn(1024, 1024, device=device)
    print("Tensor allocated on MPS:", x.device)

In [6]:
# Check cuda available
print(torch.cuda.is_available())

True


Load AG news datasets

In [7]:
label_map = {
    0: "world",
    1: "sports",
    2: "business",
    3: "sci/tech"
}
ag_news_imbalanced_data_99_to_1 = pd.read_parquet("Data/ag_news/ag_news_train_imbalanced_99_to_1.parquet")
balanced_data = pd.read_parquet("Data/ag_news/ag_news_train_balanced.parquet")
ag_news_imbalanced_data_49_to_1 = pd.read_parquet("Data/ag_news/ag_news_train_imbalanced_49_to_1_ratio.parquet")

balanced_data["label"] = balanced_data["label"].map(label_map)
ag_news_imbalanced_data_99_to_1["label"] = ag_news_imbalanced_data_99_to_1["label"].map(label_map)
ag_news_imbalanced_data_49_to_1["label"] = ag_news_imbalanced_data_49_to_1["label"].map(label_map)

# Shuffle the dataset
ag_news_imbalanced_data_99_to_1 = ag_news_imbalanced_data_99_to_1.sample(frac=1).reset_index(drop=True)
balanced_data = balanced_data.sample(frac=1).reset_index(drop=True)
ag_news_imbalanced_data_49_to_1 = ag_news_imbalanced_data_49_to_1.sample(frac=1).reset_index(drop=True)

# Testing
balanced_data



,text,label
0,"From Behind the Scenes, Rice on Stage Administ...",world
1,National union leaders set a timeline for chan...,business
2,"Brown Struggles Early, but Yankees End It Fast...",sports
3,Chelsea v Porto Defender Wayne Bridge is in co...,sports
4,NATO seeks to boost image in Middle East Middl...,world
...,...,...
995,US Stocks Up Slightly After Ford Forecast NEW...,business
996,Irish women roll by Duke Jacqueline Batteast s...,sports
997,Dollar Slumps After Weak U.S. Data NEW YORK (...,business
998,Medical Journal Calls for a New Drug Watchdog ...,world


Load toxic text dataset (The dataset is already imbalanced)

- Rename the columns to have the same names as other datasets
- Label = 0 --> Not toxic
- Label = 1 --> Toxic
- Map label column into words

In [8]:
toxic_label_map = {
    0: "nontoxic",
    1: "toxic"
}

def split_ratio_for_toxic_dataset(df, majority=None, minority=None):
    toxic_half_1 = df[df["label"] == 'nontoxic'].sample(majority, random_state=42)
    toxic_half_2 = df[df["label"] == 'toxic'].sample(minority, random_state=42)
    toxic_balanced = pd.concat([toxic_half_1, toxic_half_2], ignore_index=True, sort=False)
    toxic_balanced = toxic_balanced.sample(frac=1).reset_index(drop=True)
    return toxic_balanced

toxic_text = pd.read_csv("Data/toxic_text/train.csv")
toxic_text = toxic_text[["comment_text", "toxic"]]
toxic_text = toxic_text.rename(columns={"comment_text": "text", "toxic": "label"})
toxic_text["label"] = toxic_text["label"].map(toxic_label_map)

# Get 3 small subsets of the main datasets with 3 different ratios
toxic_balanced = split_ratio_for_toxic_dataset(toxic_text, 500, 500)
toxic_99_to_1 = split_ratio_for_toxic_dataset(toxic_text, 980, 20)
toxic_49_to_1 = split_ratio_for_toxic_dataset(toxic_text, 940, 20)


Load twitter emotion type dataset (This is also imbalanced)
- Create 3 small datasets, roughly 2000 rows each, with balanced, 99:1, 49:1 imbalanced ratios

In [9]:
emotion_map = {
    0: "sadness",
    1: "joy",
    2: "love",
    3: "anger",
    4: "fear",
    5: "surprise"
}
emotion_df = pd.read_parquet("Data/twit/twitter_emotion.parquet")
emotion_df["label"] = emotion_df["label"].map(emotion_map)

def split_ratio_for_emotion_dataset(df, majority=None, minority=None):
    imbalanced_df = []

    major_class = df[df["label"] == 'sadness'] #Assume 'sadness' is the majority label
    imbalanced_df.append(major_class.sample(majority, random_state=42))
    # Add other classes as the minority class
    for lab in list(df["label"].unique()):
        if lab != 'sadness':
            df_per_class = df[df['label'] == lab].sample(minority, random_state=42)
            imbalanced_df.append(df_per_class)
    
    imbalanced_df = pd.concat(imbalanced_df)
    imbalanced_df = imbalanced_df.sample(frac=1).reset_index(drop=True)

    return imbalanced_df


# Get 3 small subsets of the main datasets with 3 different ratios
emotion_balanced = split_ratio_for_emotion_dataset(emotion_df, majority=200, minority=200)
emotion_imbalanced_99_to_1 = split_ratio_for_emotion_dataset(emotion_df, majority=950, minority=10)
emotion_imbalanced_49_to_1 = split_ratio_for_emotion_dataset(emotion_df, majority=202, minority=10)

emotion_imbalanced_99_to_1["label"].value_counts()

        
    


label
sadness     950
fear         10
love         10
anger        10
joy          10
surprise     10
Name: count, dtype: int64

Function to build instruction for the LLMs, which can be fit with all 3 classification datasets

In [10]:
def build_prompt(df, text, label_map, shots_per_class=None):
    """
    Function to construct an instruction for the LLM

    Args:
        text (str): The text of the data

    Returns:
        prompt (str): The constructed prompt for the LLM
    """
    assert shots_per_class is not None, "Please provide 'shots_per_class' parameter"
    prompt = (
        f"You are a powerful, precise, and helpful assistant that classifies text into well-defined categories, NO MATTER THE CONTEXT."
        f" IMPORTANT: CHOOSE ONE WORD FROM THESE CATEGORIES: {', '.join(list(label_map.values()))}."
        f" Respond with exactly one word: the single best category inside the given categories, DO NOT ANSWER ANY OTHER CATEGORIES BESIDES THE GIVEN ONE."
        f" Do not explain your choice, provide reasoning, or output anything else."
        f" Learn from these examples to understand context and edge cases: "

    )
    # ASSUME THE shots_per_class WILL ALWAYS BE PASSED
    few_shots_example = []
    for lab in list(label_map.values()):
        samples = df[df['label'] == lab].sample(shots_per_class, random_state=42)
        for _, r in samples.iterrows():
            few_shots_example.append({'text': r['text'],
                                      'label': r["label"]})

    prompt += "\n\n"
    for ex in few_shots_example:
        # print(ex)
        prompt += f"Review: \"{ex['text']}\"\nCategory: {ex['label']}\n\n"
    prompt += f"Review: \"{text}\"\nCategory:" #Leave Category here blank since we want the LLM to generate text
    return prompt


# Testing function
print(build_prompt(ag_news_imbalanced_data_99_to_1, "Astros Rally Past the Giants With one swing of the bat, Lance Berkman revived the Houston Astros' playoff hopes - and gave the Los Angeles Dodgers a much-needed reprieve. Berkman hit a three-run homer off Dustin Hermanson, highlighting a five-run ninth inning that sent Houston to a 7-3 win over San Francisco on Thursday night...", label_map, shots_per_class=4))

You are a powerful, precise, and helpful assistant that classifies text into well-defined categories, NO MATTER THE CONTEXT. IMPORTANT: CHOOSE ONE WORD FROM THESE CATEGORIES: world, sports, business, sci/tech. Respond with exactly one word: the single best category inside the given categories, DO NOT ANSWER ANY OTHER CATEGORIES BESIDES THE GIVEN ONE. Do not explain your choice, provide reasoning, or output anything else. Learn from these examples to understand context and edge cases: 

Review: "Australia's plan for vast sea security zone angers neighbour New Zealand (Canadian Press) Canadian Press - WELLINGTON, N.Z. (AP) - Australia has angered its neighbour New Zealand, with a plan to impose an 1,850-kilometre security zone around the Australian coastline."
Category: world

Review: "Iran Hangs Three Drug Smugglers in Public - Paper  TEHRAN (Reuters) - Iran has hanged three drug smugglers in  a public square in the southern province of Kerman, the  Jomhuri-ye Eslami newspaper reported 

In [11]:
def clean_time(time):
  """
  Function to clean the time into prettier format, returns the better format of time
  """
  if time <= 60:
    return f"{time} seconds."

  minutes = time // 60
  remain_sec = time - minutes * 60
  return f"{minutes} minutes, {remain_sec:.2f} seconds."


Label normalization using semantic similarity
- Since we have 3 datasets, using manual variation map will not ensure every predictions that the LLM makes
- So we load a sentence embedding model to calculate the nearest vector amongst the label using Cosine Similarity

In [12]:
from sentence_transformers import SentenceTransformer, util

# Testing with AG News

embedding_model = SentenceTransformer("all-MiniLM-L6-v2")

# Ag news
valid_labs_ag_news = list(label_map.values())
valid_embeddings_ag_news = embedding_model.encode(valid_labs_ag_news, convert_to_tensor=True)

# Toxic Text
valid_labs_toxic_text = list(toxic_label_map.values())
valid_embeddings_toxic_text = embedding_model.encode(valid_labs_toxic_text, convert_to_tensor=True)

# Twitter Emotion
valid_labs_emotion = list(emotion_map.values())
valid_embeddings_emotion = embedding_model.encode(valid_labs_emotion, convert_to_tensor=True)

def normalize(label, valid_embeddings, valid_labs):
    pred_emb = embedding_model.encode(label, convert_to_tensor=True)
    cos_scores = util.cos_sim(pred_emb, valid_embeddings)[0]
    closest_idx = cos_scores.argmax().item()
    return valid_labs[closest_idx]

# Testing for AG News
raw_preds = "Fucked UP"
normalized_preds = normalize(raw_preds, valid_embeddings=valid_embeddings_emotion, valid_labs=valid_labs_emotion)
print(normalized_preds)

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

...

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

sadness


In [14]:

from tqdm.auto import tqdm
from transformers import pipeline, logging
from time import time


# Load model


# CREATE A FUNCTION TO RUN CLASSFICATION
def classify(model, df, label_map, shots, batch_size=16, max_new_tokens=3, valid_embeddings=None, valid_labs=None):
    """
    Function to run classification with different number of shots

    Args:
        model (str): name of the model
        tokenizer
        df (pd.DataFrame): the pandas dataframe
        batch_size (int): batch size per run

    Returns:
        pred_arr (List[str]): the array that contains all predictions
    """
    # Initiate a pipeline for each dataset
    # USE text2text-generation for the gemma model
    # USE text-generation for the others, or text-classification
    # USE fill-mask for distillbert
    pipe = pipeline("text-generation", model=model, dtype=torch.float16)
    logging.set_verbosity_error()

    # Generate prompts for all rows
    prompts = [build_prompt(df, text, label_map, shots_per_class=shots) for text in df["text"]]

    # Run the pipeline for each row
    pred_arr = []
    start_time = time()

    for i in range(0, len(prompts), batch_size):
        batch = prompts[i:i + batch_size] #slices a sublist of prompts
        results = pipe(batch, max_new_tokens=max_new_tokens, do_sample=False)
        for prompt, res in zip(batch, results):
            pred = res[0]['generated_text'][len(prompt):].strip().lower().split()
            # print(f"Real value: {df["label"]}")
            # print(f"Predicted value: {pred}")
            if pred[0] not in list(label_map.values()):
                normalized_pred = normalize(pred[0], valid_embeddings=valid_embeddings, valid_labs=valid_labs)
                pred_arr.append(normalized_pred)
            else:
                pred_arr.append(pred[0]) #Use pred[0] for some cases
    end_time = time()

    total_time = clean_time(end_time - start_time)

    print("Total running time is " + total_time)
    return pred_arr



Function to compute all metrics

In [ ]:
# Pass list(df["text"]) for y_true
# list(label_map.values())

def eval_llm(y_true, y_pred, label_map):
    y_true_arr = np.array([x.lower().strip() for x in y_true])
    print(y_pred)
    y_pred_arr = np.array([x.lower().strip() for x in y_pred])

    labels = [lab.lower() for lab in list(label_map.values())]

    # Calculate macro scores:
    macro_f1 = f1_score(y_true_arr, y_pred_arr, labels=labels, zero_division=0, average='macro')
    macro_recall = recall_score(y_true_arr, y_pred_arr, labels=labels, average='macro', zero_division=0)
    bal_acc = balanced_accuracy_score(y_true_arr, y_pred_arr)
    mcc = matthews_corrcoef(y_true_arr, y_pred_arr)

    # Calculate AUPRC per class
    y_true_bin = label_binarize(y_true_arr, classes=labels)
    y_pred_bin = label_binarize(y_pred_arr, classes=labels)
    if len(labels) == 2 and y_true_bin.shape[1] == 1:
        y_true_bin = np.hstack([1 - y_true_bin, y_true_bin])
        y_pred_bin = np.hstack([1 - y_pred_bin, y_pred_bin])


    auprc_per_class = {}
    for idx, cls in enumerate(labels):
        ap = average_precision_score(y_true_bin[:, idx], y_pred_bin[:, idx])
        auprc_per_class[cls] = float(ap)

    return {
        "macro_f1": float(macro_f1),
        "macro_recall": float(macro_recall),
        "balanced_accuracy": float(bal_acc),
        "mcc": float(mcc),
        "auprc_per_class": auprc_per_class
    }

    

NOW FOCUSING ON QWEN2.5 INSTRUCT MODEDLS

In [16]:
# model, df, label_map, shots, batch_size=8, max_new_tokens=3
def run_experiments(model, datasets_dict, dataset_name, label_map, shots_list=[2,4,8], batch_size=16, valid_embeddings=None, valid_labs=None):
    results = []
    for ds_name, df in datasets_dict.items():
        test_df = df.sample(frac=1).reset_index(drop=True)
        for shots in shots_list:
            preds = classify(model, test_df, label_map, shots=shots, batch_size=batch_size, valid_embeddings=valid_embeddings, valid_labs=valid_labs)
            metrics = eval_llm(test_df['label'].tolist(), preds, label_map=label_map)
            row = {
                "model": model,
                "dataset": ds_name,
                "shots": shots,
                **metrics
            }
            results.append(row)
            # persist intermediate results
            pd.DataFrame(results).to_csv(f"results/{dataset_name}/few_shot_results_{model.replace('/','_')}.csv", index=False)
    return pd.DataFrame(results)

In [17]:
# Create a dataset dict for easy mapping

ag_news_datasets_dict = {
    "ag_news_balanced": balanced_data,
    "ag_news_imbalanced_data_99_to_1": ag_news_imbalanced_data_99_to_1,
    "ag_news_imbalanced_data_49_to_1": ag_news_imbalanced_data_49_to_1
}

toxic_datasets_dict = {
    "toxic_text": toxic_balanced,
    "toxic_99_to_1": toxic_99_to_1,
    "toxic_49_to_1": toxic_49_to_1
}

emotion_datasets_dict = {
    "emotion_df": emotion_balanced,
    "emotion_imbalanced_99_to_1": emotion_imbalanced_99_to_1,
    "emotion_imbalanced_49_to_1": emotion_imbalanced_49_to_1
}

Run models + evals

In [18]:
model = "Qwen/Qwen2.5-1.5B-Instruct"

In [20]:
# RUN AG NEWS DATASET

res_df = run_experiments(model, ag_news_datasets_dict, 'ag_news',label_map, shots_list=[2,4,8], valid_embeddings=valid_embeddings_ag_news, valid_labs=valid_labs_ag_news)

config.json:   0%|          | 0.00/660 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/3.09G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/242 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

Device set to use cuda:0


Total running time is 3.0 minutes, 14.47 seconds.
['world', 'business', 'business', 'business', 'sci/tech', 'world', 'sports', 'business', 'sports', 'sports', 'world', 'business', 'business', 'business', 'business', 'sports', 'sci/tech', 'sports', 'sci/tech', 'business', 'sports', 'sci/tech', 'world', 'sports', 'business', 'sports', 'business', 'business', 'business', 'world', 'sci/tech', 'sci/tech', 'world', 'business', 'business', 'sci/tech', 'sports', 'sci/tech', 'world', 'sci/tech', 'sports', 'sports', 'world', 'world', 'business', 'world', 'sports', 'sports', 'sports', 'sports', 'business', 'sports', 'business', 'business', 'sci/tech', 'business', 'business', 'sports', 'sports', 'sci/tech', 'business', 'sports', 'sci/tech', 'sci/tech', 'sports', 'business', 'sci/tech', 'business', 'sci/tech', 'business', 'business', 'business', 'world', 'world', 'sci/tech', 'sports', 'world', 'world', 'sports', 'sports', 'business', 'sci/tech', 'business', 'business', 'sports', 'sports', 'sci/tech

In [ ]:
# RUN TOXIC TEXT DATASET
res_df = run_experiments(model, toxic_datasets_dict, "toxic_text",toxic_label_map, shots_list=[2,4,8], valid_embeddings=valid_embeddings_toxic_text, valid_labs=valid_labs_toxic_text)

In [ ]:
# RUN TWITTER EMOTION DATASET
res_df = run_experiments(model, emotion_datasets_dict, 'twitter_emotion', emotion_map, shots_list=[2,4,8], valid_embeddings=valid_embeddings_emotion, valid_labs=valid_labs_emotion)

Total running time is 9.0 minutes, 18.04 seconds.
['sadness', 'sadness', 'anger', 'sadness', 'curiosity', 'joy', 'sadness', 'sadness', 'joy', 'sadness', 'sadness', 'sadness', 'sadness', 'sadness', 'sadness', 'sadness', 'sadness', 'sadness', 'sadness', 'sadness', 'anger', 'sadness', 'sadness', 'sadness', 'joy', 'sadness', 'sadness', 'sadness', 'sadness', 'surprise', 'sadness', 'sadness', 'sadness', 'sadness', 'joy', 'joy', 'sadness', 'sadness', 'sadness', 'curiosity', 'joy', 'sadness', 'joy', 'sadness', 'sadness', 'sadness', 'sadness', 'sadness', 'sadness', 'sadness', 'sadness', 'sadness', 'sadness', 'sadness', 'sadness', 'sadness', 'sadness', 'joy', 'anger', 'sadness', 'sadness', 'sadness', 'sadness', 'sadness', 'anger', 'joy', 'sadness', 'sadness', 'sadness', 'sadness', 'sadness', 'sadness', 'joy', 'sadness', 'sadness', 'sadness', 'joy', 'sadness', 'sadness', 'sadness', 'sadness', 'sadness', 'sadness', 'sadness', 'sadness', 'fear', 'joy', 'joy', 'sadness', 'sadness', 'sadness', 'sadne

/opt/anaconda3/lib/python3.12/site-packages/sklearn/metrics/_classification.py:2480: UserWarning: y_pred contains classes not in y_true
  warnings.warn("y_pred contains classes not in y_true")


Total running time is 16.0 minutes, 51.61 seconds.
['surprise', 'surprise', 'anger', 'sadness', 'curiosity', 'joy', 'sadness', 'sadness', 'joy', 'sadness', 'sadness', 'sadness', 'surprise', 'surprise', 'surprise', 'surprise', 'sadness', 'sadness', 'surprise', 'surprise', 'surprise', 'surprise', 'sadness', 'sadness', 'surprise', 'surprise', 'sadness', 'sadness', 'surprise', 'surprise', 'surprise', 'surprise', 'surprise', 'sadness', 'joy', 'joy', 'surprise', 'sadness', 'surprise', 'curiosity', 'surprise', 'surprise', 'joy', 'sadness', 'surprise', 'surprise', 'surprise', 'sadness', 'surprise', 'surprise', 'sadness', 'sadness', 'surprise', 'sadness', 'sadness', 'sadness', 'sadness', 'joy', 'anger', 'sadness', 'sadness', 'sadness', 'sadness', 'joy', 'sadness', 'joy', 'surprise', 'sadness', 'surprise', 'sadness', 'sadness', 'sadness', 'joy', 'surprise', 'surprise', 'sadness', 'joy', 'sadness', 'sadness', 'surprise', 'sadness', 'sadness', 'sadness', 'sadness', 'surprise', 'sadness', 'joy', 'j

/opt/anaconda3/lib/python3.12/site-packages/sklearn/metrics/_classification.py:2480: UserWarning: y_pred contains classes not in y_true
  warnings.warn("y_pred contains classes not in y_true")


Total running time is 34.0 minutes, 52.58 seconds.
['surprise', 'surprise', 'surprise', 'surprise', 'surprise', 'surprise', 'surprise', 'surprise', 'surprise', 'surprise', 'surprise', 'surprise', 'surprise', 'surprise', 'surprise', 'surprise', 'surprise', 'surprise', 'surprise', 'surprise', 'surprise', 'surprise', 'surprise', 'surprise', 'surprise', 'surprise', 'surprise', 'surprise', 'surprise', 'surprise', 'surprise', 'surprise', 'surprise', 'surprise', 'surprise', 'surprise', 'surprise', 'surprise', 'surprise', 'surprise', 'surprise', 'surprise', 'surprise', 'surprise', 'surprise', 'surprise', 'surprise', 'surprise', 'surprise', 'surprise', 'surprise', 'surprise', 'surprise', 'surprise', 'surprise', 'surprise', 'surprise', 'surprise', 'surprise', 'surprise', 'surprise', 'surprise', 'surprise', 'surprise', 'surprise', 'surprise', 'surprise', 'surprise', 'surprise', 'surprise', 'surprise', 'surprise', 'surprise', 'surprise', 'surprise', 'surprise', 'surprise', 'surprise', 'surprise', 

/opt/anaconda3/lib/python3.12/site-packages/sklearn/metrics/_classification.py:2480: UserWarning: y_pred contains classes not in y_true
  warnings.warn("y_pred contains classes not in y_true")


Total running time is 39.0 minutes, 20.43 seconds.
['surprise', 'surprise', 'surprise', 'surprise', 'surprise', 'surprise', 'surprise', 'surprise', 'surprise', 'surprise', 'surprise', 'surprise', 'surprise', 'surprise', 'surprise', 'surprise', 'surprise', 'surprise', 'surprise', 'surprise', 'surprise', 'surprise', 'surprise', 'surprise', 'surprise', 'surprise', 'surprise', 'surprise', 'surprise', 'surprise', 'surprise', 'sadness', 'surprise', 'surprise', 'surprise', 'surprise', 'surprise', 'surprise', 'surprise', 'surprise', 'surprise', 'surprise', 'surprise', 'surprise', 'surprise', 'surprise', 'surprise', 'surprise', 'surprise', 'surprise', 'surprise', 'surprise', 'surprise', 'surprise', 'surprise', 'surprise', 'surprise', 'surprise', 'surprise', 'surprise', 'surprise', 'surprise', 'surprise', 'surprise', 'surprise', 'surprise', 'surprise', 'surprise', 'surprise', 'surprise', 'surprise', 'surprise', 'surprise', 'surprise', 'surprise', 'surprise', 'surprise', 'surprise', 'surprise', '